In [ ]:
import tensorflow as tf

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("/content/gdrive/MyDrive/content/FlowersDataset/")

Found 5290 files belonging to 4 classes.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/gdrive/MyDrive/content/FlowersDataset/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/14350958832_29bdd3a254.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/14402451388_56545a374a_n.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/2632216904_274aa17433.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/5054771689_00dd40b971_n.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/3750771898_cfd50090ba_n.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/3338077096_3a8ed0e2bc_m.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/2862944799_45bc8e7302.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/4565255237_9ba29c4d4e_n.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/16401288243_36112bd52f_m.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/2488902131_3417698611_n.jpg
/content/gdrive/MyDrive/content/FlowersDataset/train/daisy/14167534527_781ceb1b7a_

In [ ]:
os.listdir('/content/gdrive/MyDrive/content/FlowersDataset/train')

['daisy', 'rose', 'dandelion', 'tulip', 'sunflower']

In [ ]:
import cv2
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Data preprocessing
def preprocess_data(DIR):
    X = []
    Y = []
    labels = os.listdir(DIR)
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)

    for folder_name in labels:
        for img_name in os.listdir(os.path.join(DIR, folder_name)):
            img = cv2.imread(os.path.join(DIR, folder_name, img_name))
            img = cv2.resize(img, (150, 150))
            X.append(img)
            Y.append(folder_name)

    X = np.array(X) / 255.0  # Normalize pixel values
    Y = label_encoder.transform(Y)
    Y = to_categorical(Y, num_classes=len(labels))  # One-hot encoding

    return X, Y

trainx, trainy = preprocess_data('/content/gdrive/MyDrive/content/FlowersDataset/train')
labels = os.listdir('/content/gdrive/MyDrive/content/FlowersDataset/train')
testx, testy = preprocess_data('/content/gdrive/MyDrive/content/FlowersDataset/test')
labels = os.listdir('/content/gdrive/MyDrive/content/FlowersDataset/test')

# Model definition
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer to reduce overfitting
model.add(Dense(len(labels), activation='softmax'))  # Output layer with softmax

# Model compilation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
model.fit(trainx, trainy, epochs=10, batch_size=32)  # Increase epochs for better convergence

# Model evaluation
loss, accuracy = model.evaluate(testx, testy)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
86/86 [==============================] - 144s 2s/step - loss: 1.3767 - accuracy: 0.3995
Epoch 2/10
86/86 [==============================] - 150s 2s/step - loss: 1.1620 - accuracy: 0.5055
Epoch 3/10
86/86 [==============================] - 146s 2s/step - loss: 1.0105 - accuracy: 0.5998
Epoch 4/10
86/86 [==============================] - 143s 2s/step - loss: 0.8800 - accuracy: 0.6595
Epoch 5/10
86/86 [==============================] - 143s 2s/step - loss: 0.7311 - accuracy: 0.7243
Epoch 6/10
86/86 [==============================] - 147s 2s/step - loss: 0.6402 - accuracy: 0.7513
Epoch 7/10
86/86 [==============================] - 146s 2s/step - loss: 0.5280 - accuracy: 0.8044
Epoch 8/10
86/86 [==============================] - 141s 2s/step - loss: 0.3874 - accuracy: 0.8565
Epoch 9/10
86/86 [==============================] - 145s 2s/step - loss: 0.3464 - accuracy: 0.8736
Epoch 10/10
48/48 [==============================] - 22s 447ms/step - loss: 0.1546 - accuracy: 0.9586
Test Lo